# Побудова силуету акули з кривих Безьє 5-го порядку

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Завантаження зображення
image_path = "shark_template.png"
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Обробка: бінаризація
_, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

# Пошук контурів
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# Вибір найбільшого контуру
main_contour = max(contours, key=cv2.contourArea)

# Спрощення
epsilon = 0.005 * cv2.arcLength(main_contour, True)
approx = cv2.approxPolyDP(main_contour, epsilon, True)
approx_points = approx[:, 0, :].astype(float)

# Нормалізація
h, w = img.shape
scaled_points = (approx_points - [w // 2, h // 2]) / 20.0


In [ ]:
# Розбиття на сегменти для Безьє 5-го порядку
def chunk_closed(points, chunk_size=6):
    total = len(points)
    segments = []
    for i in range(0, total, chunk_size):
        if i + chunk_size <= total:
            segments.append(points[i:i+chunk_size])
        else:
            remaining = points[i:]
            needed = chunk_size - len(remaining)
            segment = np.vstack([remaining, points[:needed]])
            segments.append(segment)
    return segments

segments = chunk_closed(scaled_points, 6)


In [ ]:
from Bezier import Bezier

t_vals = np.linspace(0, 1, 300)

fig, ax = plt.subplots(figsize=(7, 7))
ax.set_aspect('equal')
ax.grid(True, linestyle='--', alpha=0.3)
ax.set_title("Силует акули з кривих Безьє 5-го порядку")

for idx, seg in enumerate(segments):
    curve = Bezier.Curve(t_vals, seg)
    ax.plot(curve[:, 0], -curve[:, 1], color='blue')
    midpoint = Bezier.Point(0.5, seg)
    ax.text(midpoint[0], -midpoint[1], f"S{idx+1}", fontsize=8, color='darkred')

plt.show()
